In [1]:
"""
Created on Thu May 17 21:01:53 2018

@author: Winham

===================基于1维CNN的ECG分类算法========================

*需要第三方工具包numpy,h5py,scikit-learn
*基于深度学习框架TensorFlow
*涉及函数的使用方法可自行查看工具包文档，baidu即可得
*ECG算法入门系列博客：https://blog.csdn.net/qq_15746879
*开源github：https://github.com/Aiwiscal

*本代码所需要的数据和标签文件来自matlab提取
*详情：https://blog.csdn.net/qq_15746879/article/details/80340958
==================================================================
"""

'\nCreated on Thu May 17 21:01:53 2018\n\n@author: Winham\n\n===================基于1维CNN的ECG分类算法========================\n\n*需要第三方工具包numpy,h5py,scikit-learn\n*基于深度学习框架TensorFlow\n*涉及函数的使用方法可自行查看工具包文档，baidu即可得\n*ECG算法入门系列博客：https://blog.csdn.net/qq_15746879\n*开源github：https://github.com/Aiwiscal\n\n*本代码所需要的数据和标签文件来自matlab提取\n*详情：https://blog.csdn.net/qq_15746879/article/details/80340958\n==================================================================\n'

In [2]:
import time
import numpy as np
import h5py as hp

* [attributeerror-module-tensorflow-has-no-attribute-interactivesession](https://stackoverflow.com/questions/41333798/attributeerror-module-tensorflow-has-no-attribute-interactivesession)
* [issue14](https://github.com/theislab/scgen/issues/14)

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from sklearn.metrics import confusion_matrix

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
sess=tf.InteractiveSession()

In [5]:
def load_mat(path_data,name_data,dtype='float32'):
    """
    载入.mat文件的函数,h5py解码并转换为numpy数组
    """
    data=hp.File(path_data)
    arrays_d={}
    for k,v in data.items():
        arrays_d[k]=np.array(v)
    dataArr=np.array(arrays_d[name_data],dtype=dtype)
    return dataArr

In [6]:
def CNNnet(inputs,n_class):
    """
    使用TensorFlow组件完成CNN网络的搭建，与教程中参数略有不同
    """
    
    conv1 = tf.layers.conv1d(inputs=inputs, filters=4, kernel_size=31, strides=1, \
                             padding='same', activation = tf.nn.relu)
    avg_pool_1 = tf.layers.average_pooling1d(inputs=conv1, pool_size=5, strides=5, \
                                         padding='same')
    conv2 = tf.layers.conv1d(inputs=avg_pool_1, filters=8, kernel_size=6, strides=1,\
                             padding='same', activation = tf.nn.relu)
    avg_pool_2 = tf.layers.average_pooling1d(inputs=conv2, pool_size=5, strides=5,\
                                         padding='same')
    
    flat = tf.reshape(avg_pool_2, (-1, int(250/5/5*8)))
    
    logits=tf.layers.dense(inputs=flat, units=n_class, activation=None)
    logits=tf.nn.softmax(logits)
    return logits

In [7]:
def get_batch(train_x,train_y,batch_size):
    """
    随机获取一个batch大小的数据，用于训练
    """
    indices=np.random.choice(train_x.shape[0],batch_size,False)
    batch_x=train_x[indices]
    batch_y=train_y[indices]
    return batch_x,batch_y